In [1]:
import pandas as pd
import time
print(pd.get_option('display.max_rows'))
#显示所有行 默认为60   None
pd.set_option('display.max_rows', 100)

#显示所有列
#pd.set_option('display.max_columns', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',200)

60


# Todolist
## 广告操作数据

在清理时的潜在问题，广告状态未知的都是新建广告，而这些数据都删除了，是否会有影响？
[目前位置](#训练模型了)

# 清洗数据


## totalExposureLog.out - 历史曝光日志数据
此部分每一条代表一个曝光记录。由于初赛数据规模的限制，这里每个只提供了最终竞价胜出曝光的广告信息。同一条请求，可能会同时曝光多个广告位的广告，因此一条请求id可能对应多条记录。修改拆分思路为每天分成三个时间段，将历史日志拆分。统计曝光量的时候需要结合每天的三个文件进行统计或者统计时也做追加写的方式。

为避免数据量过大，历史曝光数据选择了在用户维度（uv）上按 512 分之一进行均匀采样。各字段使用制表符(\t)分隔，每列的具体含义如下：
- 广告请求 id：唯一标识每次请求（每个请求对应一个用户某一时刻，可能多个广告位）
- 广告请求时间：该字段为时间戳，即 1970 纪元后经过的浮点秒数
- 广告位 id：加密后无业务含义，只区分不同广告位，每个广告位只能曝光特定素材尺寸的广告
- 用户 id（即看广告的人）：加密后无业务含义，只区分不同用户，可和后面的用户特征数据中 id 相关联
- 曝光广告 id：加密后无业务含义，只区分不同广告，可以和广告特征文件中的广告 id 关联
- 曝光广告素材尺寸：枚举型取值，不同广告位对素材的尺寸要求不同，同一个广告位可能适配多个不同尺寸的素材
- 曝光广告出价 bid：这里只记录 cpc 出价，非 cpc 广告此处记录折算后的 cpc 价格
- 曝光广告 pctr：预估的 pctr，和 bid 相乘得到basic_ecpm
- 曝光广告quality_ecpm：将广告质量和用户体验等因素折算成ecpm 的分数，主要影响因素有 pctr/pcvr/窄定向等
- 曝光广告totalEcpm：广告排序的分数依据，由 basic_ecpm 和quality_ecpm相加得到

起止时间20190216225954——20190319235844
for i in range(102):
    try:
        exposureLogfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\map\\totalExposureLog'+str(i)+'.out ',sep='\t',
                              header=None,names=['请求id','请求时间','广告位id','用户id','曝光广告id'
                                                            ,'素材尺寸','出价bid','pctr','quality_ecpm','totalEcpm'])
        exposureLogdf = pd.DataFrame(exposureLogfile)
        exposureLogdf['请求时间']=exposureLogdf['请求时间'].apply(lambda x:time.strftime("%Y%m%d%H%M%S",time.localtime(x))) 
        print("#"+str(i)+"文件起止时间"+exposureLogdf['请求时间'].min()+"----"+exposureLogdf['请求时间'].max())
    except StopIteration:
        break 

### 第一个100万数据中有 15427个重复，剩余984573条
exposureLogfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\map\\totalExposureLog1.out ',sep='\t',
                              header=None,names=['请求id','请求时间','广告位id','用户id','曝光广告id'
                                                            ,'素材尺寸','出价bid','pctr','quality_ecpm','totalEcpm']
                             ,nrows=1000000)
exposureLogdf= pd.DataFrame(exposureLogfile)
exposureLogdf['请求时间']=exposureLogdf['请求时间'].apply(lambda x:time.strftime("%Y%m%d%H%M%S",time.localtime(x))) #localtime
exposureLogdf.drop_duplicates(keep=False,inplace=True)
### 第一个100万数据中有 15427个重复，剩余984573条
#exposureLogdf.sort_values(by=["曝光广告id","广告位id","出价bid"])
### 请求id有935438个不同的取值，大部分一次请求对应一个广告，只有5万左右的曝光广告涉及到一个请求多个广告
### 请求时间 有85284个不同的取值，也就是每个请求时间s内都有10多次不同的访问（并发访问）
### 广告位id 251个 不同的取值
### 用户id  282381 个不同的取值
### 曝光广告id  38392个不同的取值，平均曝光不到3次
### 素材尺寸 46 个不同的取值
### 出价bid 5540 个不同的取值
### pctr  96964个不同的取值
### quality_ecpm 98005个不同的取值
### totalEcpm 519283个不同的取值
print(exposureLogdf.广告位id.unique().size)
print(exposureLogdf.用户id.unique().size)
print(exposureLogdf.曝光广告id.unique().size)
print(exposureLogdf.素材尺寸.unique().size)
print(exposureLogdf.出价bid.unique().size)
print(exposureLogdf.pctr.unique().size)
print(exposureLogdf.quality_ecpm.unique().size)
print(exposureLogdf.totalEcpm.unique().size)

In [ ]:
exposureLogfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\totalExposureLog.out ',sep='\t',
                              header=None,names=['请求id','请求时间','广告位id','用户id','曝光广告id'
                                                            ,'素材尺寸','出价bid','pctr','quality_ecpm','totalEcpm']
                             ,iterator=True,low_memory =False)
for i in range(300):
    try:
        exposureLogdf = exposureLogfile.get_chunk(1000000)
        
        #去重
        exposureLogdf.drop_duplicates(keep='first',inplace=True)        
        exposureLogdf['请求时间']=exposureLogdf['请求时间'].apply(lambda x:time.strftime("%Y%m%d%H%M%S",time.localtime(x))) 
        exposureLogdf['请求时间1']=exposureLogdf['请求时间'].apply(lambda x:(str(x[:8])+'-'+str(int(x[8:10])//8))) 
        #追加模式，每天分成三个时段0-7 8-15 16-23
        for name,group in exposureLogdf.groupby(by=['请求时间1']):
            # 追加写模式
            group.to_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\map\\totalExposureLog' + name+'.csv', 
                         sep='\t', header=False, index=False
                         ,columns=['请求id','请求时间','广告位id','用户id','曝光广告id' ,'素材尺寸','出价bid','pctr'
                                   ,'quality_ecpm','totalEcpm']
                         ,mode='a'
                        )
        print("第"+str(i)+"次读取完成！")
    except StopIteration:
        break

## user_data -- 用户特征数据
此部分包含抽样后的用户特征维度（包括上述曝光日志中全部用户），出于脱敏的考虑，这里把所有属性值（包括年龄）都做了随机映射，此部分无需过多关注id背后的业务含义。
注：用户特征字段未知的均使用 0 表示。每列特征取值都用加密后的 id 表示，均为随机映射。不同列的 id 取值区间会重复。各字段使用制表符(\t)分隔，每列的具体业务含义如下：
- 用户 id：此处和上面曝光日志文件中的用户 id 关联
- 年龄（Age）：每个取值随机映射为[1-N]的唯一 id
- 性别(Gender)：男/女
- 地域(area)：每个省/市用唯一 id 标识，可能多标签，使用逗号分隔不同 id
- 婚恋状态（Status）：单身/已婚等状态，可能去多值，使用逗号分隔
- 学历(Education)：博士/硕士/本科/高中/初中/小学
- 消费能力（ConsuptionAbility）：高/低m
- 设备（device）：IOS/Android,  不区分版本号
- 工作状态（work）：在校大学生/商旅人士/政府公职人员/科研教育者/ IT 互联网工作者/医护工作者, 可能取多值，逗号分隔
- 连接类型(ConnectionType)：无线/2G/3G/4G
- 行为兴趣(behavior)：每个兴趣点一个 id，可多值，逗号分隔

In [ ]:
# 用户特征数据 待清洗

In [267]:
userdatafile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\user_data ',sep='\t',
                              header=None,names=['用户id','年龄','性别','地域','婚恋状态'
                                                            ,'学历','消费能力','设备','工作状态','连接类型','行为兴趣']
                             ,iterator=True,low_memory =False)
count=0
for i in range(100000):
    try:
        userdatadf = userdatafile.get_chunk(10000)
        count+=userdatadf.shape[0]
    except StopIteration:
        break
print(count)

1396718


In [266]:
userdatadf.describe()

,用户id,年龄,性别,学历,消费能力,设备,连接类型
count,1.000000e+04,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,6.970675e+05,523.898500,2.49780,4.759300,2.153600,2.187900,2.812400
std,4.045921e+05,281.267209,0.56624,2.086384,0.832031,0.395739,1.022698
min,3.310000e+02,0.000000,1.00000,1.000000,1.000000,0.000000,1.000000
25%,3.410058e+05,333.000000,2.00000,2.000000,1.000000,2.000000,2.000000
50%,6.929085e+05,522.000000,3.00000,5.000000,2.000000,2.000000,2.000000
75%,1.050297e+06,787.000000,3.00000,6.000000,3.000000,2.000000,4.000000
max,1.396536e+06,988.000000,3.00000,8.000000,3.000000,4.000000,5.000000



## test_sample.dat -- 测试数据集
此部分包含需要预估广告的全部设置条件，包含新创建广告、已创建广告、修改过及未修改修改过的广告。预估目标是下一个自然日（0点到24点）的曝光量。历史数据为3月20日（不含）之前，因此对于 20日以后新建广告，其预估目标为预估创建次日，对于已存在广告预估目标为3月21日（周四）

第 n+1 天的待预估广告以及对应的设置。各列由制表符(\t)分隔， 含义说明如下：
-  样本 id
-  广告 id
-  创建时间
-  素材尺寸
-  广告行业 id
-  商品类型
-  商品 id
-  广告账户 id
-  投放时段
-  人群定向
-  出价（单位分）

以上各字段的格式均和训练数据中的广告数据格式一致。
由于要评估出价相关性，故测试数据中一条广告 id 会对应多条不同出价的样本。为简化问题，测试数据也要求预估指定广告样本在下一个自然日的曝光量。（此处的曝光量是和训练数据一样 uv 采样后的结果）。为了更准确的评估，此处的测试广告已经剔除掉下一个自然日期间再次修改的广告（如暂停、修改出价、定向等），即认为在预估周期内该广告设置保持不变。

In [124]:
# 测试数据   (20290, 11)
test_samplefile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\test_sample.dat ',sep='\t',
                              nrows=30000,header=None,names=['样本id','广告id','创建时间','素材尺寸','广告行业id'
                                                           ,'商品类型','商品id','广告账户id','投放时间','人群定向'
                                                            ,'出价'])
test_sampledf = pd.DataFrame(test_samplefile)

In [125]:
# 拆分投放时间，扩展字段  
# 周一到周四，每天不投放的都有 10个  # 周六日 周六不投放的62个   周日不投放的145个
test_sampledf=pd.merge(test_sampledf,test_sampledf['投放时间'].str.split(',',expand=True),right_index=True,left_index=True)

# 20190321000000 对应时间戳为 1553097600   在此之前的样本有17809 个，需要判断周四是否在投放时间内，也就是第三个，之后是否是全0
# 结论：周四不投放的有10个，这10个曝光应该为0  

# 10个样本的id为 [ 6008,  6468,  7917,  9962, 10194, 12376, 14849, 15552, 16017,18827]
(test_sampledf[(test_sampledf[3]=='0')&(test_sampledf['创建时间']<1553097600)]['样本id']).values


对于3月21日0点之后新建的广告，预估曝光的投放时间为创建的时候的第二天，第二天不在投放时间的有20个左右，样本id为 

[ 1103,  4826,  5548,  5591,  6079,  8175,  9845,  9876, 10238,
       11280, 11429, 11590, 11758, 12649, 14507, 15044, 15984, 16640,
       17850, 19113, 20289],

In [ ]:
#把时间列标准化时间格式 输出这一天是周中的第几天，Monday=0, Sunday=6
test_sampledf['dayofweek']=(pd.to_datetime(test_sampledf['创建时间'],unit ='s')+pd.offsets.Hour(8)+pd.DateOffset(days=1)).dt.dayofweek
# +(24+8)*3600  
# 5, 6, 4, 0
temp=test_sampledf[(test_sampledf['创建时间']>=1553097600)]
# temp.dayofweek.unique()
temp=temp[((temp[0]=='0') | (temp[1]=='0')| (temp[2]=='0')| (temp[3]=='0')| (temp[4]=='0')| (temp[5]=='0')| (temp[6]=='0'))]
temp['out']=temp.apply(lambda x: ('0' if x[x['dayofweek']]=='0' else '1'),axis=1)
temp[temp['out']=='0']['样本id'].values  #这20个也要设置曝光为0

# 共涉及到三个广告id   77259  571523  667503

In [199]:
#test_sampledf['创建时间'].max() 
# 最小时间  1433409092 2015/6/4 17:11:32
# 最大时间  1553413953  2019/3/24 15:52:33日

# 取值种类统计   
# 广告id：1954 素材尺寸：20（没有逗号分隔） 广告行业id:125   商品类型:10    
# 商品id有：872个值   广告账户id：1104  投放时间:222  人群定向：476个  出价：490 (范围1-20000)
# test_sampledf.出价.unique().max() #.size
# gp=test_sampledf.groupby(by=['广告id','素材尺寸','广告行业id' ,'商品类型','商品id','广告账户id','投放时间','人群定向'])
# 1954 个，证明了同一个广告的 '广告id','素材尺寸','广告行业id' ,'商品类型','商品id','广告账户id' ,'投放时间','人群定向' 不变
# gp['广告id','素材尺寸','广告行业id' ,'商品类型','商品id','广告账户id','投放时间','人群定向'].count()

# 清空
#temp.drop(temp.index,inplace=True)
#print(test_sampledf.shape)
#print(test_sampledf.dtypes)

## ad_static_feature.out - 广告静态数据
此部分除最后一列素材尺寸的缺失值使用空字符表示外，其他列的缺失值是用-1表示。关于逗号连接的多值数据，本文件中只有手册中提到的素材尺寸（即最后一列）是合法的，其他列出现的不合理数据需进行数据清洗。

该类广告属性一般从广告创建后无法修改。所有 id 类数据均为加密后随机映射。各列用制表符分隔，含义如下：
-  广告 id：和曝光日志中的广告 id 相关联
-  创建时间：广告创建时的时间戳
-  广告账户 id：广告所在账户的唯一标识，账户结构分为四级：账户——推广计划——广告——素材
-  商品 id：广告推广目标的唯一标识，若推广目标是落地页，则该字段为空
-  商品类型：广告推广目标的类型，枚举型
-  广告行业 id：广告所属的行业类别标识
-  素材尺寸：不同广告位对素材的尺寸要求不同，同一个广告可能有多个不同尺寸的素材，用逗号分隔

In [152]:
# 分析广告静态数据  735911行数据 
adstaticfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\ad_static_feature.out ',sep='\t',
                              nrows=10000000,header=None,names=['广告id','创建时间','广告账号id','商品id','商品类型'
                                                            ,'广告行业id','素材尺寸']
                          ,low_memory=False)
adstaticdf = pd.DataFrame(adstaticfile)
#adstaticdf['创建时间']=adstaticdf['创建时间'].apply(lambda x:time.strftime("%Y%m%d%H%M%S",time.gmtime(x))) #localtime
#adstaticdf

In [153]:
# 无重复数据
adstaticdf.drop_duplicates(keep='first',inplace=True)

#广告行业id 包含,分隔，需要清洗  8191 rows  剩余727720条
#adstaticdf['广告行业id'].str.contains(',')
adstaticdf=adstaticdf[~adstaticdf['广告行业id'].str.contains(',')]
#adstaticdf['广告行业id']=pd.to_numeric(adstaticdf['广告行业id'])
adstaticdf['广告行业id']=adstaticdf['广告行业id'].astype('int64')

#商品id 包含,分隔，需要清洗  3 rows  剩余 727717 条
#adstaticdf[adstaticdf['商品id'].str.contains(',')]
adstaticdf=adstaticdf[~adstaticdf['商品id'].str.contains(',')]
adstaticdf['商品id']=pd.to_numeric(adstaticdf['商品id'])

# 创建时间为0的有 9195 rows 剩余 718522   条  
# 最新考虑不能删，作为历史数据考虑把
#adstaticdf=adstaticdf[(adstaticdf['创建时间']!=0)]

# 素材尺寸为NaN的有 220856  rows 太多了，别删了吧.
# 官方说了删除，剩余497666万
adstaticdf= adstaticdf[~(adstaticdf['素材尺寸'].isnull())]

adstaticdf['创建时间']=adstaticdf['创建时间'].apply(lambda x:time.strftime("%Y%m%d%H%M%S",time.localtime(x)))
adstaticdf.sort_values(by=['广告id','创建时间'])
adstaticdf.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\ad_static_feature_p.csv",sep="\t",
                     index=False,encoding="utf-8")

## ad_operation.dat - 广告的操作数据
此部分第二列操作时间为0的视为缺失，对于创建操作，创建时间可以在ad_staic_feature.out得到。其他不合理的数据需要进行清洗。

记录广告操作的流水数据，以及操作后的属性值。各列使用制表符分隔，含义如下：
-  广告 id（同上）
-  创建/修改时间: 即广告创建或者修改设置的时间
-  操作类型：1-修改，2-新建
-  修改字段：1-广告状态，2-出价，3-人群定向，4-广告时段设置
-  操作后的字段值：
-  广告状态取值：1- 正常，0-失效
-  出价：整数（单位分）
-  投放时段：字符串。

包含 7 个 64 位无符号整型数字（逗号分隔），每个整数分别代表周一到周日的投放时段。该整数转为 2 进制后从低到高 48 位
bit 代表全天各时段（半小时为一时间窗口）是否投放，1-投放，0-不投。举例说明，17179865088= 1111111111111111111111000000000000，代表投放时段为 6：00-17：00；281474976710655=111111111111111111111111111111111111111111111111，代表全天投放。

- 人群定向：字符串。格式如下：

feature_name1:feature_value1,feature_value2|feature_name2:feature_value3,feature_value4|… 
此处 feature_name 取值同用户属性文件中的各列属性名，feature_value 取值 id 同用户属性文件中的定义，不同feature 用“|”分隔，不同 feature 取值用逗号分隔。
广告通过人群定向的设置来召回对应的用户请求，对应的人群规则：不同 feature_name 是求 交 集 ， 同 一 featurename 下 不 同 的 value 求并集 ， 未 定 义 的feature_name 则表示 该维度 不 限 。 举 例 如：定向 设 置 为age:51,62,73,84|gender:1|area:1,3,5 ; 则表示该广告能被“（年龄 id 为51 或 62 或 73 或 84） 且 （性别取值为 1） 且 （地域取值为 1 或 3或 5）”的用户召回（即在这些用户上有曝光机会）。

### 读取


In [154]:
# 分析广告操作数据  760866行数据 其中647381为修改，其余为新建，新建的时间为0
adoperationfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\ad_operation.dat ',sep='\t',
                             nrows=1000000,header=None,names=['广告id','创建时间','操作类型','修改字段','操作后的状态值']
                            ,dtype = {'创建时间' : str})
adoperationdf = pd.DataFrame(adoperationfile)
#adoperationdf['创建时间']=adoperationdf['创建时间'].astype(str)
#adoperationdf.loc[adoperationfile['创建时间']!=0]  

In [543]:
# adoperationdf[adoperationdf['操作类型']==2]

### 删除重复值

In [155]:
# 把重复的值删除 326 rows/2
adoperationdf.drop_duplicates(keep='first',inplace=True)
#print(adoperationdf[adoperationdf.duplicated(keep='first')])

# 清理创建时间为0、操作类型为修改的 1522 row记录   剩余759181  rows
adoperationdf=adoperationdf[~((adoperationdf['创建时间']=='0')&(adoperationdf['操作类型']==1))]

# 删除不合法日期 20190230000000  1292 rows  剩余 757889 条
adoperationdf=adoperationdf[(adoperationdf['创建时间']!="20190230000000")]

# 同一广告、同一创建时间、同一操作类型，同一修改字段修改为不同的值 1202行重复数据,剩余756687 条
# 发现这类数据都是密集交替修改状态，之前和之后还有秒级差别的修改记录，重复的删除
adoperationdf.drop_duplicates(subset=['广告id','创建时间','操作类型','修改字段'],keep=False,inplace=True)
#print(adoperationdf[adoperationdf.duplicated(subset=['广告id','创建时间','操作类型','修改字段'],keep=False)])


### 把相关字段修改值转成列

In [156]:
#把相关字段修改值转成列
adoperationdf['index11']=adoperationdf.apply(lambda x: str(x['广告id'])+'-'+str(x['创建时间'])+'-'+str(x['操作类型']),axis=1)
#print(adoperationdf.index11.unique().size)  # 680131个  
# 剩余680131个，还需要把这两个再关联起来
#adoperationdf.pivot(index='index11', columns='修改字段', values='操作后的状态值')
temp=adoperationdf.pivot(index='index11', columns='修改字段', values='操作后的状态值')
temp.rename(index=str, columns={1: "广告状态", 2: "出价",3:"人群定向",4:"投放时间"},inplace=True) 

#已经行转列了，可以清理重复数据
adoperationdf.drop_duplicates(subset=['广告id','创建时间','操作类型'],keep='last',inplace=True)
# 将行转列的数据与原数据合并
adoperationdf=pd.merge(adoperationdf,temp,how='left',left_on='index11',right_on='index11')
# 清理无用字段
adoperationdf.drop(['修改字段','操作后的状态值','index11'],axis=1,inplace=True)

adoperationdf.sort_values(['广告id','创建时间'],inplace=True)

adoperationdf.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\ad_operation_p.csv",sep="\t",
                     index=False,encoding="utf-8")
#20190216000000
#adoperationdf[adoperationdf['创建时间']!="0"]['创建时间'].min()
# 20190319235959
#adoperationdf['创建时间'].max()
temp.drop(temp.index,inplace=True)

### 填充缺失值

In [413]:
#adoperationfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\ad_operation_p.csv ',sep='\t',nrows=1000000,dtype = {'创建时间' : str})
#adoperationdf = pd.DataFrame(adoperationfile)
# 填充缺失值？？ 有错误，必须按照分组填充，不能跨广告id填充
#adoperationdf.fillna(axis=0,method='ffill',inplace=True)
# 有问题
#for name,group in adoperationdf.groupby(by=['广告id']):
#    group.fillna(axis=0,method='ffill',inplace=True)

# 按分组划分，笨方法进行添加
temp=adoperationdf.groupby(by=['广告id'])
tempfill=temp.fillna(axis=0,method='ffill')

#tempfill 列顺序乱了，增加一列广告id
tempfill['广告id']=adoperationdf['广告id']
order = ['广告id', '创建时间',"操作类型","广告状态","出价","人群定向","投放时间"]
adoperationdf = tempfill[order]
adoperationdf.sort_values(by=["广告id","创建时间"],inplace=True)

adoperationdf.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\ad_operation_p.csv",sep="\t",
                     index=False,encoding="utf-8")

### 删除广告状态无效行

In [165]:
#62637  行为空的数据  广告状态为nan的   剩余617494数据
#adoperationdf[adoperationdf['广告状态'].isnull()]
adoperationdf=adoperationdf[~adoperationdf['广告状态'].isnull()]
adoperationdf.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\ad_operation_p1.csv",sep="\t",
                     index=False,encoding="utf-8")

### 清理频繁修改记录
只保留一天的最后一条修改记录

In [166]:
# 按小时为单位统计出 235538条频繁修改记录
#adoperationdf['创建时间h']=adoperationdf['创建时间'].apply(lambda x: x[:10])

# 训练集需要以天为预估单位 523645 条每天多余两次的修改操作，保留当天最后一次修改记录
#在预估第二天时还要看第二天是否有修改记录，第二天有修改记录的需要剔除
adoperationdf['创建时间d']=adoperationdf['创建时间'].apply(lambda x: x[:8])
#adoperationdf[adoperationdf.duplicated(subset=['广告id','创建时间d','操作类型'],keep=False)]
adoperationdf.drop_duplicates(subset=['广告id','创建时间d','操作类型'],keep='last',inplace=True)
# 剩余270221 条记录
# 清理无用字段 ,'创建时间h'
adoperationdf.drop(['创建时间d'],axis=1,inplace=True)
adoperationdf.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\ad_operation_p2.csv",sep="\t",
                     index=False,encoding="utf-8")


In [93]:
# 查看空值情况
# print(adoperationdf.dtypes)
# adoperationdf[adoperationdf['广告id']<1]
# 查看操作类型是否合法数据  
# adoperationdf[~adoperationdf['操作类型'].isin([1, 2])]
# adoperationdf[~adoperationdf['修改字段'].isin([1,2,3,4])]
# 创建时间是否异常
# adoperationdf[adoperationdf['创建时间']=="20190230000000"]
# adoperationdf[(adoperationdf['创建时间'].str.len()!=1)&(adoperationdf['创建时间'].str.len()!=14)]
#创建时间为0的可以在ad_staic_feature.out得到
# pd.to_datetime(adoperationdf['创建时间'])
#adoperationdf['创建时间1']=pd.to_datetime(adoperationdf['创建时间'],errors="ignore")
#adoperationdf[(adoperationdf['创建时间1']==0)&(adoperationdf['创建时间']!=0)]
#adoperationdf.drop(['创建时间1'],axis=1,inplace=True)


# adoperationdf[adoperationdf['操作后的状态值'].isnull()]
# adoperationdf[(adoperationdf['修改字段']==1)&(adoperationdf['操作后的状态值']!="0")&(adoperationdf['操作后的状态值']!="1")]
# pd.to_numeric(adoperationdf[(adoperationdf['修改字段']==2)]['操作后的状态值'])
#adoperationdf[adoperationdf['操作后的状态值'].str.count("\|")>0]

# 构造训练集

## 思路
首先清洗收据，按照官方给的标准将无效数据剔除掉，上一步已经完成了。
根据操作记录表，内连接广告静态数据，只有静态数据没有操作记录的广告舍弃无法用于训练，没有参考特征用于训练。上一步已完成

需要把广告操作记录中的最终值聚合到各字段投放时间、人群定向、出价、状态开时间、状态关闭时间、下次变动时间 小于该行createtime的数据中最大的createtime，修改自动类型相同、广告id一致，变动后作为一个新广告进行统计。上一步对该思路进行了调整已完成。

## 训练集字段
['样本id','广告id','创建时间','素材尺寸','广告行业id','商品类型','商品id','广告账户id','投放时间','人群定向','出价']
从20190217到20190319日曝光数量作为label，按天进行训练

### 出价
特征
### 素材尺寸
 按照题目说明会有多个，多个的维度如何考虑？多个特征？拆分开，没有的算作0？  该特征对pctr有较大影响。

经过计算未发现多值，取值有
['40', nan, '30', '34', '1', '64', '14', '20', '41', '54', '48', '55', '33', '12', '32', '37',
'25', '63', '66', '4', '16', '11', '56', '44', '31']
### 广告账户id、商品id、商品类型、广告行业id
作为特征用于训练，可能和广告质量有关，用曝光数据中pctr、ecpm进行训练测试？


## 对广告静态数据和操作数据进行整合
生成和测试集一样的数据集
### 读取清洗过的操作数据和静态数据

In [169]:
# temp.drop(temp.index,inplace=True)
# 读取清洗后的广告操作和静态数据    270221 行操作数据，502153行静态数据
adoperationfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\ad_operation_p2.csv ',sep='\t',
                              nrows=1000000,dtype = {'创建时间' : str}
                              ,low_memory=False)
adoperationdf = pd.DataFrame(adoperationfile) #270221 行数据

In [170]:
adstaticfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\ad_static_feature_p.csv ',sep='\t',
                              nrows=10000000,dtype = {'创建时间': str},low_memory=False)
adstaticdf = pd.DataFrame(adstaticfile)  #502153 行数据

### 合并静态数据和操作数据，以操作数据为主表
清理多余字段，调整字段顺序

In [174]:
# 合并静态数据和操作数据，以操作数据为主表，左连接 (680131 , 12)
mergedf=pd.merge(adoperationdf, adstaticdf, how='left', left_on='广告id', right_on='广告id')
# 对于没有创建时间的操作数据取 静态数据中的时间，之后还没有时间的默认为很久以前
mergedf['创建时间']=mergedf.apply(lambda x: (x['创建时间_y'] if x['创建时间_x']=='0' else x['创建时间_x']),axis=1)
# 删除多余字段
mergedf.drop(['创建时间_x', '创建时间_y'], axis=1,inplace=True)
#调整列顺序  ['样本id','广告id','创建时间','素材尺寸','广告行业id','商品类型','商品id','广告账户id','投放时间','人群定向','出价']
order = ['广告id', '创建时间','素材尺寸','广告行业id', '商品类型', '商品id', '广告账号id', '出价','人群定向','投放时间', '操作类型', '广告状态']
mergedf = mergedf[order]
mergedf.sort_values(by=["广告id","创建时间"],inplace=True)

#先写入文件以免丢失数据
mergedf.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\adoperationleftjoinadstatic.csv",sep="\t",
                     index=False,encoding="utf-8")

### 清理素材尺寸为空的数据

In [175]:
# 其中 145284 素材尺寸未知 
#mergedf[mergedf.素材尺寸.isnull()==True]
#mergedf.素材尺寸.unique()

# 对合并后的数据进行再次清理，缺失广告静态数据会影响pctr等计算  剩余 218265  
mergedf.dropna(subset=['素材尺寸'],inplace=True) 

### 按日生成训练样本数据

In [177]:
#获取某日可供训练的样本
# 需要依次生成 0216到0318的每日可供训练样本（adoperationdf的子集），用于预测次日的曝光数据（0217-0319）
#20190216225954——20190319235844

predatelist=['20190217000000','20190218000000','20190219000000','20190220000000','20190221000000','20190222000000',
             '20190223000000','20190224000000','20190225000000','20190226000000','20190227000000','20190228000000',
             '20190301000000','20190302000000','20190303000000','20190304000000','20190305000000','20190306000000',
             '20190307000000','20190308000000','20190309000000','20190310000000','20190311000000','20190312000000',
             '20190313000000','20190314000000','20190315000000','20190316000000','20190317000000','20190318000000',
             '20190319000000','20190320000000']

In [194]:
for i in range(len(predatelist)-1):
    #预测0217日曝光为例，需要找出在0217零点前已经创建的广告
    #adoperationdf[((adoperationdf['创建时间']<'20190217000000')&(adoperationdf['广告状态']=='1'))]
    # 如果有多条记录的话，只获取最近的广告设置  按照广告id分组获取最近的一条
    temp=mergedf[((mergedf['创建时间']<predatelist[i])
                        &(mergedf['广告状态']==1))].groupby(by=['广告id']).tail(1)

    # 要求该广告在第二天没有修改记录，否则影响训练
    nextdayadlist=mergedf[((mergedf['创建时间']>=predatelist[i]) &
                                 (mergedf['创建时间']<predatelist[i+1]) )].广告id.unique()

    # 删除第二天有改动广告，不作为训练样本
    temp=temp[((~temp['广告id'].isin(nextdayadlist) ))] 
    # 样本不为空则写入文件中作为训练样本
    if not (temp.empty):
        # 清理无用字段
        temp.drop(['操作类型','广告状态'],axis=1,inplace=True)
        temp.reset_index(inplace=True,drop=True)
        
        #字段顺序
        order = ['广告id', '创建时间','素材尺寸','广告行业id','商品类型','商品id','广告账号id','投放时间','人群定向','出价']
        temp=temp[order]
        
        #字段类型 广告行业id 商品类型 商品id 广告账号id是字符串，不带小数点 
        temp['广告账号id']=temp['广告账号id'].astype('int64')
        temp['商品类型']=temp['商品类型'].astype('int64')
        temp['商品id']=temp['商品id'].astype('int64')
        temp['广告行业id']=temp['广告行业id'].astype('int64')
        temp.to_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\test_sample_' + predatelist[i]+'.csv',
                    sep='\t', header=False)

## 曝光次数 label统计
统计在曝光记录中该广告id对应时间范围内的曝光次数，按照天统计每日曝光量。
（ 该次数需要按照广告的有效投放时间按照7*24小时换算成天为单位）？这个思路可能不对

In [ ]:
# 确认需要将历史曝光日志数据按照时间统计每日曝光次数，用于训练时预测每个广告第二天的曝光量
# 对拆分后的曝光数据进行分析
# 读取按条数分隔的原始数据
# exposureLogfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\map\\totalExposureLog2.out ',sep='\t',
#                               header=None,names=['请求id','请求时间','广告位id','用户id','曝光广告id','素材尺寸'
#                                                  ,'出价bid','pctr','quality_ecpm','totalEcpm'])
# exposureLogdf = pd.DataFrame(exposureLogfile)
# exposureLogdf['请求时间']=exposureLogdf['请求时间'].apply(lambda x:time.strftime("%Y%m%d%H%M%S",time.localtime(x))) 

# 按照时间和广告id排序，统计每个广告的次数
#exposureLogdf.sort_values(by=["广告位id","曝光广告id","出价bid"])
#exposureLogdf.sort_values(by=["请求时间","曝光广告id"],inplace=True)
# gp=exposureLogdf.groupby(by=['曝光广告id','请求时间']) # '素材尺寸',
# newdata=gp['曝光广告id','请求时间'].count()

# newdata.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\exposureByDay.csv",sep="\t",index=False,encoding="utf-8")

### 对曝光数据进行按天汇总

In [677]:
#temp  #1022807 171
#print(exposurestatistics.shape)
#temp=temp.reset_index().groupby(by=['曝光广告id','出价'])['请求id'].sum().reset_index()
#temp.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\exposureByDay000000.csv",sep="\t",header=False,encoding="utf-8")

In [195]:
#对拆分后的日志文件进行遍历，统计每日广告曝光数据
filenameprefix='totalExposureLog'
datelist=['20190217','20190218','20190219','20190220','20190221','20190222',
             '20190223','20190224','20190225','20190226','20190227','20190228',
             '20190301','20190302','20190303','20190304','20190305','20190306',
             '20190307','20190308','20190309','20190310','20190311','20190312',
             '20190313','20190314','20190315','20190316','20190317','20190318',
             '20190319']
for datestr in datelist:
    filename=filenameprefix+datestr+'-'  
    temp.drop(temp.index,inplace=True)
    for i in range(3):
        exposureLogfile =pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\map\\' + filename+str(i)+'.csv',sep='\t',
                                      header=None,names=['请求id','请求时间','广告位id','用户id','曝光广告id' ,'素材尺寸','出价','pctr'
                                           ,'quality_ecpm','totalEcpm'])

        exposureLogdf=pd.DataFrame(exposureLogfile)
        # 曝光广告id和素材尺寸好像是1对1的关系
        # 一个广告针对不同的用户有不同的pctr、quality_ecpm 等
        # 同一个广告一天多次不同报价的怎么处理？  应该没有影响，毕竟训练样本中已经排除掉这类广告，或者这里增加一个出价统计
        exposurestatistics=exposureLogdf.groupby(by=['曝光广告id'])['请求id'].count().reset_index()
        #print(exposurestatistics.shape)
        if temp.empty:
            temp=exposurestatistics
        else:
            temp=pd.concat([temp,exposurestatistics])
    temp=temp.reset_index().groupby(by=['曝光广告id'])['请求id'].sum().reset_index()
    temp.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\exposureByDay"+datestr+".csv",sep="\t",header=False,index=False,encoding="utf-8")

In [198]:
temp

,曝光广告id,请求id


### 结合训练样本来分析对应label


In [197]:
temp.drop(temp.index,inplace=True) 
#(exposureLogdf.groupby(by=['曝光广告id','出价','quality_ecpm'])['请求id'].count())


In [199]:
# 某日0点之前的广告样本，用于计算某日的广告曝光数据 ,文件名集合
predatelist=['20190218000000','20190219000000','20190220000000','20190221000000','20190222000000',
             '20190223000000','20190224000000','20190225000000','20190226000000','20190227000000','20190228000000',
             '20190301000000','20190302000000','20190303000000','20190304000000','20190305000000','20190306000000',
             '20190307000000','20190308000000','20190309000000','20190310000000','20190311000000','20190312000000',
             '20190313000000','20190314000000','20190315000000','20190316000000','20190317000000','20190318000000',
             '20190319000000']

# 曝光统计数据 文件名集合
datelist=['20190218','20190219','20190220','20190221','20190222',
             '20190223','20190224','20190225','20190226','20190227','20190228',
             '20190301','20190302','20190303','20190304','20190305','20190306',
             '20190307','20190308','20190309','20190310','20190311','20190312',
             '20190313','20190314','20190315','20190316','20190317','20190318',
             '20190319']
temp1=pd.DataFrame()
for i in range(len(predatelist)):
    print(i)
    samplefilename='test_sample_'+predatelist[i]+'.csv'
    test_samplefile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\'+samplefilename,sep='\t',
                                  header=None,names=['样本id','广告id', '创建时间','素材尺寸','广告行业id','商品类型',
                                                     '商品id','广告账号id','投放时间','人群定向','出价'])
    test_sampledf = pd.DataFrame(test_samplefile)

    exposureLogfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\exposureByDay'+datelist[i]+'.csv',sep='\t',
                                  header=None,names=['广告id','曝光次数'])
    exposureLogdf = pd.DataFrame(exposureLogfile)
    # 将出价合并统计，否则会出现一对多的情况，导致样本变化。已经合并过了
    #exposureLogdf=exposureLogdf.groupby(by=['广告id'])['曝光次数'].sum().reset_index()
    temp=pd.merge(test_sampledf,exposureLogdf,how='left',left_on='广告id',right_on='广告id')
    
    # 处理空值
    temp['曝光日期']=datelist[i]
    #temp['出价_y']=temp.apply(lambda x: ( x['出价_x'] if pd.isnull(x['出价_y']) else x['出价_y']),axis=1)
    temp.fillna(0,inplace=True)
    if temp1.empty:
        temp1=temp           
    else:
        temp1=pd.concat([temp1,temp])
    #temp.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\sample\\sample"+datelist[i]+".csv",sep="\t",header=False,index=False,encoding="utf-8")
print('ok')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
ok


In [235]:
temp1.groupby(by=['样本id','广告id', '创建时间','素材尺寸','广告行业id','商品类型',
                  '商品id','广告账号id','投放时间','人群定向','出价'
                  ,'曝光日期'])['曝光次数'].sum().reset_index().sort_values(['曝光日期'
                                                                     ,'广告id']).to_csv(
    "E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\sample\\sample_total.csv",sep="\t",index=False,encoding="utf-8")

In [259]:
#temp2=temp1.groupby(by=['广告id'])['曝光次数'].sum().reset_index()
#emptyadids=temp2[temp2['曝光次数']==0]['广告id'].unique()
#temp2=temp1[~ temp1['广告id'].isin(emptyadids)]

temp2=temp1[temp1['曝光次数']==0].groupby(by=['广告id'])['曝光次数'].count().reset_index()
emptyadids=temp2[temp2['曝光次数']>7]['广告id'].unique()
temp2=temp1[~ (temp1['广告id'].isin(emptyadids)&(temp1['曝光次数']==0) )]

In [260]:
temp2.describe()

,样本id,广告id,创建时间,素材尺寸,广告行业id,商品类型,商品id,广告账号id,出价,曝光次数
count,47257.000000,47257.000000,4.725700e+04,47257.000000,47257.000000,47257.000000,47257.000000,47257.000000,47257.000000,47257.000000
mean,3885.102017,360956.386715,2.019029e+13,28.599192,113.795438,8.913833,10744.965994,15176.620141,123.073026,31.065218
std,3034.088904,212057.711681,3.870029e+07,19.049003,67.500158,5.778198,10938.830615,8647.309160,266.181199,247.783678
min,0.000000,332.000000,2.019022e+13,1.000000,-1.000000,1.000000,-1.000000,16.000000,10.000000,0.000000
25%,1310.000000,176466.000000,2.019023e+13,12.000000,47.000000,1.000000,-1.000000,7749.000000,50.000000,0.000000
50%,3101.000000,357244.000000,2.019031e+13,30.000000,122.000000,13.000000,7610.000000,14999.000000,88.000000,1.000000
75%,6010.000000,542297.000000,2.019031e+13,40.000000,156.000000,13.000000,20523.000000,22673.000000,136.000000,9.000000
max,11516.000000,736023.000000,2.019032e+13,66.000000,251.000000,18.000000,32911.000000,29742.000000,10000.000000,18923.000000


In [261]:
# 将30天汇总后曝光次数仍为0的数据删掉
temp2.to_csv("E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\sample\\sample_total_no0.csv",sep="\t",index=False,encoding="utf-8")


# 训练模型了
## 先不考虑投放时间、投放人群的影响
## 生成对应表格
## 按照曝光次数取平均值，然后提交看结果

In [34]:

temp1.groupby(['广告id'])['曝光次数'].mean()

In [57]:
#temp['出价_y']=
temp1.sort_values(['广告id','出价_x','出价_y'])
temp2=temp1.drop_duplicates(keep='first',subset=['广告id','出价_y','曝光次数'])

In [62]:
#temp1.reset_index(inplace=True)
#temp2.drop(['样本id','创建时间','素材尺寸','广告行业id','商品类型','商品id','广告账号id','投放时间','人群定向','出价_x'],axis=1,inplace=True)
#.drop_duplicates(keep='first',inplace=True)
temp2.sort_values(['广告id','出价_y'],inplace=True)
temp2[temp2['曝光次数']!=0.0]

d:\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,广告id,出价_y,曝光次数
0,415,300.0,2.0
0,415,300.0,3.0
0,415,300.0,9.0
1,415,300.0,1.0
1,415,300.0,6.0
1,415,600.0,1.0
1,415,600.0,2.0
2,415,600.0,7.0
2,415,600.0,4.0
2,415,600.0,3.0


In [ ]:
# 包含 7 个 64 位无符号整型数字（逗号分隔），每个整数分别代表周一到周日的投放时段。
# 该整数转为 2 进制后从低到高 48 位 bit 代表全天各时段（半小时为一时间窗口）是否投放，1-投放，0-不投。
# 举例说明，17179865088= 1111111111111111111111000000000000，代表投放时段为 6：00-17：00；
# 281474976710655=111111111111111111111111111111111111111111111111，代表全天投放。
#把数字转换成二进制，并拆分出每一位，逆序后形成一个List
# t = '17179865088,281474976710655'
# print(Period2List (t))
def long2bit (n):
    pass
    #转成二进制,前面再补0到48位
    t = bin(n)[2:].zfill(48)
    lstRet = list(t)
    lstRet.reverse()
    return lstRet

#把投放时段转换成List,二层; 
#投放时段：字符串。包含 7 个 64 位无符号整型数字（逗号分隔）
#测试：
# t =  "17179865088,17179865088,4294950912,17109865088,17109865088,281474976710655,281474976710655"
# t = '17179865088,281474976710655'
#print(Period2List (t))  
def Period2List (strPeriod):
    pass
    lstT = strPeriod.split(',')
    lstR = []
    for x in lstT:
        lstR.append(long2bit(int(x)))
    return lstR

# 流量分析
对曝光数据进行流量分析，分析每天的每半小时流量情况、各素材尺寸展现的概率、各广告行业的展现概率、各商品类型的展现概率等。。。


In [143]:
adstaticfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\ad_static_feature.out ',sep='\t',
                              nrows=10000000,header=None,names=['广告id','创建时间','广告账号id','商品id','商品类型'
                                                            ,'广告行业id','素材尺寸']
                          ,low_memory=False)
adstaticdf = pd.DataFrame(adstaticfile)
# 无重复数据
adstaticdf.drop_duplicates(keep='first',inplace=True)

#广告行业id 包含,分隔，需要清洗  8191 rows  剩余727720条
#adstaticdf['广告行业id'].str.contains(',')
adstaticdf=adstaticdf[~adstaticdf['广告行业id'].str.contains(',')]
#adstaticdf['广告行业id']=pd.to_numeric(adstaticdf['广告行业id'])
adstaticdf['广告行业id']=adstaticdf['广告行业id'].astype('int64')

#商品id 包含,分隔，需要清洗  3 rows  剩余 727717 条
#adstaticdf[adstaticdf['商品id'].str.contains(',')]
adstaticdf=adstaticdf[~adstaticdf['商品id'].str.contains(',')]
adstaticdf['商品id']=pd.to_numeric(adstaticdf['商品id'])

In [145]:
exposureLogfile = pd.read_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\totalExposureLog.out ',sep='\t',
                              header=None,names=['请求id','请求时间','广告位id','用户id','广告id'
                                                            ,'素材尺寸','出价bid','pctr','quality_ecpm','totalEcpm']
                             ,iterator=True,low_memory =False)
for i in range(120):
    try:
        exposureLogdf = exposureLogfile.get_chunk(1000000)
         #去重
        exposureLogdf.drop_duplicates(keep='first',inplace=True)        
        exposureLogdf['请求时间']=exposureLogdf['请求时间'].apply(lambda x:time.strftime("%Y%m%d%H%M%S",time.localtime(x))) 
        exposureLogdf['请求时间1']=exposureLogdf['请求时间'].apply(lambda x:(str(x[:10])+'-'+str(int(x[10:12])//30))) 
        temp=pd.merge(exposureLogdf,adstaticfile,how='left',left_on='广告id',right_on='广告id')
        temp['素材尺寸_x']=temp['素材尺寸_x'].astype("str")

        # 统计半小时的总请求数
        totalcount=pd.DataFrame()
        totalcount=temp.groupby(by=['请求时间1'])['请求id'].nunique().reset_index() #distinct
        #total[total['请求时间1']=='2019021623-1']['请求id'].get_values()[0]
        totalcount.rename(index=str, columns={"请求时间1":"请求时间","请求id":"总次数"},inplace=True) 

        # 统计半小时 各素材尺寸的展示次数
        scalecount=pd.DataFrame()
        scalecount=(temp.groupby(by=['请求时间1','素材尺寸_x'])['请求id'].count()).reset_index() 
        scalecount.rename(index=str, columns={"请求时间1":"请求时间","素材尺寸_x": "类型","请求id":"次数"},inplace=True) 
        scalecount['类别']='scalecount'

        # 统计半小时 各商品id的展示次数
        productidcount=pd.DataFrame()
        productidcount=temp.groupby(by=['请求时间1','商品id'])['请求id'].count().reset_index()   # -1的需要考虑删除了
        productidcount.rename(index=str, columns={"请求时间1":"请求时间","商品id": "类型","请求id":"次数"},inplace=True) 
        productidcount['类别']='productidcount'

        # 统计半小时 各商品类型的展示次数
        producttypecount=pd.DataFrame()
        producttypecount=temp.groupby(by=['请求时间1','商品类型'])['请求id'].count().reset_index() 
        producttypecount.rename(index=str, columns={"请求时间1":"请求时间","商品类型": "类型","请求id":"次数"},inplace=True) 
        producttypecount['类别']='producttypecount'

        # 统计半小时 各广告行业id的展示次数
        adindustryidcount=pd.DataFrame()
        adindustryidcount=temp.groupby(by=['请求时间1','广告行业id'])['请求id'].count().reset_index()
        adindustryidcount.rename(index=str, columns={"请求时间1":"请求时间","广告行业id": "类型","请求id":"次数"},inplace=True) 
        adindustryidcount['类别']='adindustryidcount'
        
        scalecount=pd.concat([scalecount,productidcount,producttypecount,adindustryidcount])
        scalecount=pd.merge(scalecount,totalcount,how='left',left_on="请求时间",right_on="请求时间")
        # 追加写模式   columns 请求时间	类型	次数	类别	总次数
        scalecount.to_csv('E:\\work\\tencent\\algo.qq.com_641013010_testa\\testA\\halfhourExposureLog.csv', 
                          sep='\t', header=False, index=False,mode='a')
        
    except StopIteration:
        break


### 投放时间
可以考虑把7个数都换成2进制统计一共有多少个1，但这样就会丢失黄金时间段的概念，或者把全部二进制都换算成特征维度，交给模型去算系数。
7*48个特征，特征值为0-1

### 投放人群
投放人群统计，按照广告设置的投放人群在用户数据中，统计满足条件的用户数有多少（用户数/用户总人数），访问用户id也会影响到pctr